In [60]:
import time


In [61]:
## Definición de variables
examenes = [('E1', 'C1'), ('E2', 'C2'), ('E3', 'C3'), ('E4', 'C4'), ('E1', 'C5'), ('E2', 'C6'), ('E3', 'C7')]
dias = ['Lunes', 'Martes', 'Miércoles']


In [62]:
## Restricciones para garantizar que todos los exámenes se realicen en días diferentes, que
#ningún estudiante tenga más de un examen por día y que los estudiantes que toman el mismo
#curso no tengan exámenes el mismo día

def solucion_valida(asignacion, examen_actual, dia_actual):
    for examen, dia in asignacion.items(): 
        if dia == dia_actual:
            estudiante_actual, curso_actual = examen_actual
            estudiante, curso = examen
            if estudiante_actual == estudiante or curso_actual == curso:
                return False
    return True

## Algoritmo de Backtraking Search

In [24]:
# MCV Variable más restringida


def MCV(examenes, asignacion, dias):
    min_opciones = float('inf')
    examen_seleccionado = None
    for examen in examenes:
        if examen not in asignacion:
            opciones_validas = 0
            for dia in dias:
                if solucion_valida(asignacion, examen, dia):
                    opciones_validas += 1
            if opciones_validas < min_opciones:
                min_opciones = opciones_validas
                examen_seleccionado = examen
    return examen_seleccionado


def backtracking_search(examenes, dias, asignacion={}):
    if len(asignacion) == len(examenes):
        return asignacion
    
    examen = MCV(examenes, asignacion, dias)
    if examen is not None:
        for dia in dias:
            if solucion_valida(asignacion, examen, dia):
                asignacion[examen] = dia 
                resultado = backtracking_search(examenes, dias, asignacion)  
                if resultado is not None:
                    return resultado  
                asignacion.pop(examen)
    else:
        print("No se encontró solución")
    return None

In [25]:
def imprimir_solucion(solucion):
    if solucion is None:
        print("No se encontró ninguna solución.")
        return
    
    solucion_por_dia = {}
    for examen, dia in solucion.items():
        if dia not in solucion_por_dia:
            solucion_por_dia[dia] = []
        solucion_por_dia[dia].append(examen)

    for dia, examenes in solucion_por_dia.items():
        print(f"{dia}:")
        for examen in examenes:
            estudiante, curso = examen
            print(f"  Estudiante {estudiante}, Curso {curso}")
        print()

inicio = time.time()
solucion = backtracking_search(examenes, dias)
fin = time.time()

imprimir_solucion(solucion)

duracion = fin - inicio
print(f"Tiempo de ejecución: {duracion} segundos.")

Lunes:
  Estudiante E1, Curso C1
  Estudiante E2, Curso C2
  Estudiante E3, Curso C3
  Estudiante E4, Curso C4

Martes:
  Estudiante E1, Curso C5
  Estudiante E2, Curso C6
  Estudiante E3, Curso C7

Tiempo de ejecución: 0.001135110855102539 segundos.


El algoritmo de Backtracking Search pudo encontrar una solución óptima para el problema de manera muy eficiente, en un tiempo extremadamente corto, lo que sugiere que es adecuado para resolver este tipo de problemas incluso con un conjunto de datos modestamente grande.

## Beam Search Algorithm

In [58]:
def calcular_conflictos(asignacion):
    conflictos = 0
    examenes_asignados = list(asignacion.keys())
    for i, examen1 in enumerate(examenes_asignados):
        for examen2 in examenes_asignados[i + 1:]:
            dia1 = asignacion[examen1]
            dia2 = asignacion[examen2]
            if dia1 == dia2 and (examen1[0] == examen2[0] or examen1[1] == examen2[1]):
                conflictos += 1
    return conflictos

def beam_search(examenes, dias, ancho_beam=2):
    estados = [{}]
    for examen in examenes:
        nuevos_estados = []
        for estado in estados:
            for dia in dias:
                if solucion_valida(estado, examen, dia):
                    nuevo_estado = estado.copy()
                    nuevo_estado[examen] = dia
                    nuevos_estados.append(nuevo_estado)
        
        # Evaluar y mantener solo los 'ancho_beam' mejores estados
        nuevos_estados.sort(key=lambda x: calcular_conflictos(x))
        estados = nuevos_estados[:ancho_beam]
        
        # Verificar si algún estado asigna todos los exámenes
        for estado in estados:
            if len(estado) == len(examenes):
                return estado
                
    return None 

In [59]:
def imprimir_solucion(solucion):
    if solucion is None:
        print("No se encontró ninguna solución.")
        return
    
    solucion_por_dia = {}
    for examen, dia in solucion.items():
        if dia not in solucion_por_dia:
            solucion_por_dia[dia] = []
        solucion_por_dia[dia].append(examen)

    for dia, examenes in solucion_por_dia.items():
        print(f"{dia}:")
        for examen in examenes:
            estudiante, curso = examen
            print(f"  Estudiante {estudiante}, Curso {curso}")
        print()

inicio = time.time()
solucion = beam_search(examenes, dias)
fin = time.time()

imprimir_solucion(solucion)

duracion = fin - inicio
print(f"Tiempo de ejecución: {duracion} segundos.")


Lunes:
  Estudiante E1, Curso C1
  Estudiante E2, Curso C2
  Estudiante E3, Curso C3
  Estudiante E4, Curso C4

Martes:
  Estudiante E1, Curso C5
  Estudiante E2, Curso C6
  Estudiante E3, Curso C7

Tiempo de ejecución: 0.0 segundos.


El algoritmo de Beam Search ha demostrado ser eficaz para resolver el problema de programación de exámenes, asignando exámenes a diferentes días de la semana de manera que se respeten todas las restricciones impuestas, como la limitación de un examen por día para cada estudiante y la restricción de que estudiantes que toman el mismo curso no tengan exámenes el mismo día.

En cuanto al tiempo de respuesta, el algoritmo de Beam Search ha mostrado un rendimiento excelente, encontrando una solución válida en un tiempo de ejecución muy corto, en este caso, 0.0 segundos. Esto demuestra la eficiencia del algoritmo en resolver problemas de este tipo incluso con conjuntos de datos más grandes.

## Local Search Algorithm

In [63]:
import random


def generar_vecinos(solucion):
    vecinos = []
    for examen, dia in solucion.items():
        for nuevo_dia in dias:
            if nuevo_dia != dia and solucion_valida(solucion, examen, nuevo_dia):
                vecino = solucion.copy()
                vecino[examen] = nuevo_dia
                vecinos.append(vecino)
    return vecinos
    
def local_search(examenes, dias):
    solucion_actual = {examen: random.choice(dias) for examen in examenes}
    mejor_solucion = solucion_actual.copy()
    mejor_conflictos = calcular_conflictos(mejor_solucion)
    iteraciones = 0
    
    while True:
        vecinos = generar_vecinos(solucion_actual)
        print(f"Iteración: {iteraciones}, Vecinos generados: {len(vecinos)}")
        if not vecinos:
            break
        
        vecino = min(vecinos, key=calcular_conflictos)
        conflictos_vecino = calcular_conflictos(vecino)
        
        if conflictos_vecino < mejor_conflictos:
            mejor_solucion = vecino
            mejor_conflictos = conflictos_vecino
        
        if conflictos_vecino == 0:
            break
        
        solucion_actual = vecino
        iteraciones += 1
    
    print(f"Iteraciones realizadas: {iteraciones}")
    return mejor_solucion

inicio_local_search = time.time()
solucion_local_search = local_search(examenes, dias)
fin_local_search = time.time()

imprimir_solucion(solucion_local_search)
duracion_local_search = fin_local_search - inicio_local_search
print(f"Tiempo de ejecución de Local Search: {duracion_local_search} segundos.")


Iteración: 0, Vecinos generados: 8
Iteraciones realizadas: 0
Miércoles:
  Estudiante E1, Curso C1
  Estudiante E4, Curso C4

Lunes:
  Estudiante E2, Curso C2
  Estudiante E1, Curso C5
  Estudiante E3, Curso C7

Martes:
  Estudiante E3, Curso C3
  Estudiante E2, Curso C6

Tiempo de ejecución de Local Search: 0.00099945068359375 segundos.
